In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import mean_squared_error
import datetime
import xarray as xr
from dateutil.relativedelta import relativedelta
import multiprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load

df=pd.read_csv('prepare_2014-2022.csv',index_col=0)


In [ ]:
#df=df.drop(columns=["RADM2_CH4_"])
zero_counts = df.apply(lambda x: x[x == -9999.0].count())

print(zero_counts.to_csv("E:\\de.csv"))

In [ ]:
df_filtered = df[~df.isin([ -9999.0]).any(axis=1)]


In [ ]:
print(len(df_filtered))

In [ ]:

df=df_filtered

df=df.sample(n=200000)


# 使用随机森林回归模型进行拟合
y = df.iloc[:, 3]

df2 = df.drop(columns=['exact_time', 'average'])
df2 = (df2 - df2.mean()) / df2.std()
X = df2.iloc[:, :]


In [ ]:
print(X)

In [ ]:
# 创建随机森林回归模型
rf = RandomForestRegressor(n_estimators=200, random_state=42, verbose=1,n_jobs=15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

# 训练模型
rf.fit(X_train, y_train)

# 预测测试集
y_pred = rf.predict(X_test)

# 计算均方误差
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# 输出变量的贡献度
importances = rf.feature_importances_
feature_names = X.columns
sorted_indices = np.argsort(importances)[::-1]
for index, importance in enumerate(importances):
    print(f'{feature_names[sorted_indices[index]]}: {importance}')
top_indices = sorted_indices[:20]
# 可视化变量的贡献度
plt.figure(figsize=(30, 18))
plt.barh(feature_names[top_indices], importances[top_indices], color='skyblue')
plt.xlabel('Feature Importance')
plt.title('Feature Importances from Random Forest')
plt.savefig("E:\\pjy.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 输出变量的贡献度
importances = rf.feature_importances_



feature_names = X.columns


# 创建DataFrame
df_importances = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

# 按贡献度排序并选择前20个特征
df_importances_sorted = df_importances.sort_values(by='Importance', ascending=False)

# 保存为Excel文件
csv_path = 'feature_importances.csv'
df_importances_sorted.to_csv(csv_path, index=False)


sorted_indices = np.argsort(importances)[::-1]
for index, importance in enumerate(importances):
    print(f'{feature_names[sorted_indices[index]]}: {importance}')
top_indices = sorted_indices[:20]




# 可视化变量的贡献度
plt.figure(figsize=(10, 6))
plt.barh(feature_names[top_indices], importances[top_indices], color='skyblue')
plt.xlabel('Feature Importance')
plt.title('Feature Importances from Random Forest')
plt.savefig("E:\\pjy.png", dpi=300, bbox_inches='tight')
plt.show()